In [ ]:
import geopandas as gpd
import pandas as pd



In [ ]:
combined_gdf = gpd.read_feather('data/prepared/combined.feather')

In [ ]:
gdf = combined_gdf.reset_index(drop=True)

In [ ]:
combined_gdf['category'].unique()

In [ ]:
def category_mapping(category:str):
    category = category.lower()
    remap = {
        'hotel':'restaurant',
        'casual dining':'restaurant',
        'food/beverage specialty store':'grocery',
        'bakery & baked goods store': 'grocery',
        'tower': 'tourist attraction',
        'monument': 'tourist attraction',
        'sports facility/venue': 'bar or pub',
        'primary school':'primary/secondary school',
        'secondary school':'primary/secondary school',
        'special school': 'primary/secondary school'
    }
    remapped_cat = remap.get(category,category)
    return remapped_cat

# [print(f'{x} --> {category_mapping(x)}') for x in combined_gdf['category'].unique()]

gdf['category'] = gdf['category'].apply(category_mapping)

In [ ]:
gdf['category'].unique()

In [ ]:
gdf.plot()

In [ ]:
focus_zone = gpd.read_feather('data/prepared/lga.feather').unary_union

destinations_gdf = gdf[gdf.intersects(focus_zone)]

In [ ]:
ax = destinations_gdf.plot(figsize=(10, 10))
gpd.GeoSeries([focus_zone]).plot(ax=ax, color='none', edgecolor='red', linewidth=2)

In [ ]:
from pyproj import CRS
import osmnx as ox

GDA2020 = CRS(7844)

GDA2020

In [ ]:
# for origin points, we import the road network and population densities, and distribute points along the road network accordingly

from osmnx.io import save_graph_geopackage

n, s, e, w = -37.4, -38.1, 145.4, 144.4

bbox = [n,s,e,w]

road_graph = ox.graph_from_polygon(focus_zone, network_type='walk',
                                   simplify=False, retain_all=False)


save_graph_geopackage(road_graph,'data/prepared/graph.gpkg',directed=False)

nodes, edges = ox.graph_to_gdfs(road_graph, nodes=True, edges=True)




In [ ]:
import folium

center_point = focus_zone.centroid
m = folium.Map(location=[center_point.y, center_point.x], zoom_start=15)

# Add the edges and nodes to the map
# This can be resource-intensive for large datasets, consider simplifying if it's too heavy
ox.plot_graph_folium(road_graph, graph_map=m, popup_attribute='name', edge_width=2)

In [ ]:
from osmnx.io import save_graph_geopackage

In [ ]:
road_graph.nodes(data=True)

In [ ]:
nodes.head(10)

In [ ]:
edges.head(20)

In [ ]:
import igraph